In [5]:
!pip install arxiv
!pip install wikipedia

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11758 sha256=3448c192461d9942d66e620ab8a8a331e85145354bdc9e8851061715fb90925e
  Stored in directory: /home/vscode/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

### Our first tool --> if using wrapper no need for retreiver

In [9]:
## creating api wrapper for wikipedia
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

#vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())

vectordb=FAISS.from_documents(documents,OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://host.docker.internal:11434" ))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0xffff70b86530>, search_kwargs={})

### Our second tool web search

In [13]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")
retriever_tool.name

'langsmith_search'

### our 3 tool for arxiv

In [14]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [15]:
## combining all tools 
tools=[wiki,retriever_tool,arxiv]

In [ ]:
# from dotenv import load_dotenv

# load_dotenv()
# import os
# os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [ ]:
from langchain_community.llms import Ollama  # this will not work as it just model and you cant assign role like this 

# to assign role and chat history we need to make use of chat model by using ChatOllama
from langchain_community.chat_models import ChatOllama
llm=ChatOllama(model="Gemma3:1b",base_url="http://host.docker.internal:11434",temperature=0.2)

/tmp/ipykernel_5061/271892881.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm=ChatOllama(model="Gemma3:1b",base_url="http://host.docker.internal:11434",temperature=0.2)


In [28]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/home/vscode/anaconda3/envs/llm/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [29]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [30]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [33]:
agent_executor.invoke({"input":"RISCV simulation using B and K extension"})



> Entering new AgentExecutor chain...
Okay, let's dive into RISCV (Risk-Based Cybersecurity) simulations using B and K extensions. This is a really interesting and rapidly evolving area, and I can definitely help you understand the concepts and provide guidance. Here’s a breakdown of what we can cover, broken down into manageable sections:

**1. Understanding RISCV and its Role**

* **What is RISCV?** RISCV (Risk-Based Cybersecurity) is a framework designed to help organizations systematically assess and manage cybersecurity risks. It’s a process focused on prioritizing security efforts based on the *likelihood and impact* of potential threats.
* **Key Components:**
    * **Risk Assessment:** Identifying potential threats and vulnerabilities.
    * **Risk Prioritization:** Ranking risks based on their potential impact.
    * **Security Controls:** Implementing measures to mitigate risks.
    * **Monitoring & Review:** Continuously assessing the effectiveness of controls.

**2. B and 

{'input': 'RISCV simulation using B and K extension',
 'output': "Okay, let's dive into RISCV (Risk-Based Cybersecurity) simulations using B and K extensions. This is a really interesting and rapidly evolving area, and I can definitely help you understand the concepts and provide guidance. Here’s a breakdown of what we can cover, broken down into manageable sections:\n\n**1. Understanding RISCV and its Role**\n\n* **What is RISCV?** RISCV (Risk-Based Cybersecurity) is a framework designed to help organizations systematically assess and manage cybersecurity risks. It’s a process focused on prioritizing security efforts based on the *likelihood and impact* of potential threats.\n* **Key Components:**\n    * **Risk Assessment:** Identifying potential threats and vulnerabilities.\n    * **Risk Prioritization:** Ranking risks based on their potential impact.\n    * **Security Controls:** Implementing measures to mitigate risks.\n    * **Monitoring & Review:** Continuously assessing the effe